In [9]:
checkins = sc.textFile('../YelpDataset/yelp_academic_dataset_checkin.json')
businesses = sc.textFile('../YelpDataset/yelp_academic_dataset_business.json')

In [34]:
import operator
import math
import json

selectedCategories = ['Restaurants', 'Shopping', 'Beauty & Spas', 'Bars', 'Automotive', 'Event Planning & Services',
                     'Fast Food', 'Coffee & Tea', 'Hotels', 'Real Estate', 'Dentists', 'Gyms']

def mapper_checkins_filter(records):
    for line in records:
        line = json.loads(line)
        checkin_info = line['checkin_info']
        for key, val in checkin_info.iteritems():
            yield (line['business_id'], val)

def mapper_business_filter(rows):
    for row in rows:
        row = json.loads(row)
        for category in row['categories']:
            if(category in selectedCategories):
                yield (row['business_id'], category)

def mapper_normalization(records):
    for category in records:
        minCheckins = min(category[1], key=operator.itemgetter(1))[1]
        maxCheckins = max(category[1], key=operator.itemgetter(1))[1]
        for business in category[1]:
            normalizedRatingValue = (float(business[1]) - minCheckins)/(maxCheckins - minCheckins) * 20
            yield (category[0], (business[0], normalizedRatingValue))    
        
checkins_filtered = checkins.mapPartitions(mapper_checkins_filter).reduceByKey(operator.add)
business_filtered = businesses.mapPartitions(mapper_buisness_filter)
joined = checkins_filtered.join(business_filtered).map(lambda x: (x[1][1],(x[0],x[1][0]))).groupByKey().mapValues(list)
total_normalized = joined.mapPartitions(mapper_normalization).groupByKey().mapValues(list)
total_sorted = total_normalized.map(lambda (k, v): (k, sorted(v, key=lambda x: x[1], reverse=True)))
total_sorted.take(2)

[(u'Coffee & Tea',
  [(u'DWU6qqrlrC4wDkxhbHo8Tw', 20.0),
   (u'aRkYtXfmEKYG-eTDf_qUsw', 12.594258975974084),
   (u'2lt03ZLGvgPApOKS7eKGOQ', 12.066948618734815),
   (u'pwF_phwjD1ZHsY1xC9H7mA', 9.824529829928911),
   (u'bqYrEwL5xjqv0u8XAij--Q', 8.435166021776299),
   (u'S-Wd--9bLPtgfs8CtTmGuQ', 7.565913794654908),
   (u'Hgbxen9iKGL4AH6kKgC8YQ', 7.425537658598039),
   (u'YppzR-sn8y15dmUd1lLcfw', 6.5796814541527935),
   (u'59U8McUEjsRC_7h4ALk12Q', 6.318725816611176),
   (u'L9UYbtAUOcfTgZFimehlXw', 6.234140196166652),
   (u'OHsuvX4rmhnPvvRBb-QK7A', 5.615045442274813),
   (u'mWFA3S-h2_bEvjDJP-11wg', 5.39188337982543),
   (u'pmPqJ6N35WgibLWivJ0p2g', 5.211913974624314),
   (u'yXMvhCmUADlaHabo-DiUYw', 4.783586790245658),
   (u'hfpt_mEBm1ZLI2zrqfXwaA', 4.7097993341132005),
   (u'vaabk6CYXX1dYVQ1xkyPUg', 4.385854404751192),
   (u'lcq2CBlGdnVWzTDlrBAJ7w', 4.153693872041753),
   (u'BQjGE1Z3nk253XzIZqt8Gg', 3.8009538378475662),
   (u'lktu5JPDlQUG-7cV7gOzDQ', 3.7883559794834873),
   (u'jCUOqEK8Lln0PK